# 부록 10.1: 프롬프트 연결하기

- [수업](#lesson)
- [예시 실습장](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Has been rewritten to take in a messages list of arbitrary length
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## 수업

"글쓰기는 다시 쓰기다"라는 말이 있습니다. **Claude는 그렇게 하도록 요청받을 때 종종 응답의 정확성을 향상시킬 수 있습니다**!

Claude에게 "다시 생각해보라"고 프롬프트하는 방법은 여러 가지가 있습니다. 인간에게 자신의 작업을 다시 확인하도록 요청하는 것이 자연스럽게 느껴지는 방법들은 일반적으로 Claude에게도 작동할 것입니다. (프롬프트 연결을 언제 어떻게 사용할지에 대한 추가 예시는 우리의 [프롬프트 연결 문서](https://docs.anthropic.com/claude/docs/chain-prompts)를 확인하세요.)

### 예시

이 예시에서는 Claude에게 열 개의 단어를 제시하도록 요청했지만... 그 중 하나 이상이 실제 단어가 아닙니다.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

**Claude에게 답변을 더 정확하게 만들도록 요청하면** 오류가 수정됩니다! 

아래에서는 위의 Claude의 잘못된 응답을 가져와서 Claude에게 이전 답변을 수정하도록 요청하는 대화의 다른 턴을 추가했습니다.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

하지만 Claude가 우리에게 그렇게 하라고 했다고 해서 단순히 답변을 수정하고 있는 것일까요? 이미 올바른 답변으로 시작한다면 어떨까요? Claude가 자신감을 잃을까요? 여기서는 `first_response` 자리에 올바른 응답을 배치하고 다시 한 번 더 확인하도록 요청했습니다.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

위 블록에서 응답을 여러 번 생성해 보면 Claude가 대부분의 경우 단어를 그대로 두지만, 모든 단어가 이미 올바름에도 불구하고 때때로 단어를 바꾸는 것을 알 수 있습니다. 이를 완화하기 위해 무엇을 할 수 있을까요? 챕터 8에 따르면, Claude에게 탈출구를 제공할 수 있습니다! 한 번 더 시도해 봅시다.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

위 코드에서 응답을 여러 번 생성해보면 Claude가 이제 자신의 입장을 고수하는 것이 훨씬 나아진 것을 확인할 수 있습니다.

프롬프트 연결을 사용하여 **Claude에게 응답을 더 좋게 만들도록 요청**할 수도 있습니다. 아래에서는 먼저 Claude에게 이야기를 쓰도록 한 다음, 작성한 이야기를 개선하도록 요청했습니다. 개인적인 취향은 다를 수 있지만, 많은 사람들이 Claude의 두 번째 버전이 더 낫다고 동의할 것입니다.

먼저 Claude의 첫 번째 버전 이야기를 생성해 봅시다.

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

이제 Claude가 첫 번째 초안을 개선하도록 해봅시다.

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

이러한 형태의 대체는 매우 강력합니다. 우리는 목록, 단어, Claude의 이전 응답 등을 전달하기 위해 대체 자리 표시자를 사용해 왔습니다. 또한 **대체를 사용하여 우리가 "함수 호출"이라고 부르는 것을 할 수 있습니다. 이는 Claude에게 어떤 함수를 수행하도록 요청한 다음, 그 함수의 결과를 가져와서 그 결과로 Claude에게 더 많은 일을 하도록 요청하는 것입니다**. 이는 다른 대체와 마찬가지로 작동합니다. 이에 대한 자세한 내용은 다음 부록에서 다룹니다.

아래는 Claude에 대한 한 호출의 결과를 가져와서 다른, 더 긴 호출에 연결하는 또 다른 예시입니다. 첫 번째 프롬프트부터 시작해 봅시다 (이번에는 Claude의 응답을 미리 채우는 것도 포함됩니다).

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

이 이름 목록을 다른 프롬프트에 전달해 봅시다.

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

이제 프롬프트 연결에 대해 배웠으니, 프롬프트 연결을 사용하여 함수 호출을 구현하는 방법을 배우기 위해 부록 10.2로 이동하세요.

---

## 예시 실습장

이 곳은 이 수업에서 보여준 프롬프트 예시들을 자유롭게 실험해보고 프롬프트를 조정하여 Claude의 응답이 어떻게 달라지는지 확인할 수 있는 공간입니다.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))